In [3]:
%load_ext autoreload
%autoreload 2
from utils.data import load_data, get_word_tokenized_corpus, get_data_property, get_data_chunks
from utils.embeddings import train_fasttext_embedding, get_chunk_embeddings, save_fasttext, load_fasttext
from utils.features import get_speed, get_volume, get_circuitousness
from main import setup_chunk_embeddings, setup

[nltk_data] Downloading package punkt to /home/smoorjani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/smoorjani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/smoorjani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/smoorjani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
%reload_ext autoreload

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [6]:
import glob
import numpy as np
import pandas as pd
from os.path import exists
from sklearn.linear_model import Lasso

In [31]:
# Train Model
class Args:
#     model_name = 'fasttext_model/ft_model.model'
    model_name = 'fasttext_model/cc.en.300.bin'
    data_file = 'data/dblp-ref-0.json'
    chunk_embs_file = 'none'
    proj_dir = './saved/'
    limit = 30000
    T = 20
    train_model = True

args=Args()

ft_model, abstracts, citation_counts = setup(args)
chunk_embs = setup_chunk_embeddings(args, ft_model, abstracts)

Loading data...
Retrieving model...
Training model...
Chunking abstracts...


In [17]:
# SET THIS FLAG IF IMPORTING EMBS FROM MATLAB
from_MATLAB = False

# Load Chunks
class Args:
    model_name = 'fasttext_model/cc.en.300.bin'
    data_file = 'data/dblp-ref-0.json'
    chunk_embs_file = 'data/chunk_embs.txt'
    proj_dir = './saved/'
    limit = 30000
    T = 20
    train_model = False

args=Args()

In [19]:
if from_MATLAB:
    df = pd.read_table('./saved/data/toubia_embs.txt', dtype=float, header=None, sep=',').fillna(0).values
    chunk_embs = [np.trim_zeros(df[i], 'b').reshape(300,-1).transpose() for i in range(len(df))]
else:
    chunk_embs = setup_chunk_embeddings(args, None, None)

Loading chunk embeddings...


In [32]:
data = load_data(args)
abstracts = get_data_property(data, "abstract")
citation_counts = get_data_property(data, "n_citation")

In [33]:
features = {}
features['speed'] = [get_speed(chunk_emb)[-1] for chunk_emb in chunk_embs]
# features['circuitousness'] = [get_circuitousness(chunk_emb) for chunk_emb in chunk_embs]
features['volume'] = [get_volume(chunk_emb) for chunk_emb in chunk_embs]

/home/smoorjani/word_embedding_measures/utils/algs.py:62: RuntimeWarning: invalid value encountered in double_scalars
  step_size = (maximum - d - 1.0) / ((d + 1.0) * (maximum - 1.0))
/home/smoorjani/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:411: RuntimeWarning: Mean of empty slice.
  return np.exp(log_a.mean(axis=axis))
/home/smoorjani/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
print('Getting coefficients...')
for key, value in features.items():
    clf = Lasso(alpha=0.1)
    nan_vals = np.argwhere(np.isnan(features['volume']))
    
    clf.fit(np.delete(np.array(citation_counts), nan_vals).reshape(-1, 1), np.delete(value, nan_vals))
    print(f'{key} coeff {clf.coef_}')

Getting coefficients...
speed coeff [-0.00063021]
volume coeff [-0.00010053]


Getting coefficients...
speed coeff [-0.00064022]
volume coeff [-0.00021089]